## Match 30-min MRMS Composite Reflectivity Tracks to 30-min Ensemble Storm Tracks 

### Goal: Determine if a new object ID criteria produces a more skillful prediction of observed storms. 

In [1]:
from glob import glob
import xarray as xr
import numpy as np
from os.path import join
import os
from WoF_post.wofs.plotting.wofs_colors import WoFSColors
from WoF_post.wofs.plotting.wofs_levels import WoFSLevels
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
from skimage.measure import regionprops
from scipy.ndimage import maximum_filter, gaussian_filter, minimum_filter
from scripts import WoFSVerifier, add_contingency_table_compos

from skexplain.common.multiprocessing_utils import run_parallel, to_iterator

import sys
sys.path.insert(0, '/home/monte.flora/python_packages/MontePython')
import monte_python

%matplotlib inline

In [2]:
def new_id(input_data, remove_low=True):
    
    param_set = [ {'min_thresh': 0,
                   'max_thresh': 100,
                   'data_increment': 1,
                   'delta': 0,
                   'area_threshold': 600,
                   'dist_btw_objects': 125 },
             
             {'min_thresh': 30,
                   'max_thresh': 100,
                   'data_increment': 1,
                   'delta': 0,
                   'area_threshold': 200,
                   'dist_btw_objects': 5 },
            ]

    params = {'params': param_set }

    # Less than 2/18 = 0.11, 1/18 = 0.055
    new_input_data = np.copy(input_data)
    if remove_low:
        new_input_data[input_data<=0.12] = 0

    new_input_data = maximum_filter(new_input_data, size=2)
    new_input_data = gaussian_filter(new_input_data, 1)*100

    storm_labels, new_object_props = monte_python.label(  input_data = new_input_data, 
                       method ='iterative_watershed', 
                       return_object_properties=True, 
                       params = params,  
                       )
    
    # Reduce the object size due to the maximum filter and gaussian filter 
    idx = np.where(input_data==0)
    storm_labels[idx] = 0
    
    #storm_labels = minimum_filter(storm_labels, size=3)
    new_object_props = regionprops(storm_labels, storm_labels)
    
    return storm_labels, new_input_data, new_object_props

## Get the 30-min time-max MRMS composite reflectivity & the 30-min ensemble storm tracks

In [3]:
# MCS case: '20210504 / 2200-0000'
# Isolated case : '20210526'

date = '20210504'
init_time = '2300'
time_idxs = [6]
MRMS_PATH = '/work/brian.matilla/WOFS_2021/MRMS/RAD_AZS_MSH_AGG/'
WOFS_PATH = '/work/mflora/SummaryFiles/'

def get_files(date, init_time, t):
    WOFS_OFFSET = 6 
    MRMS_OFFSET = 18 

    WOFS_PATH = '/work/mflora/SummaryFiles/'
    MRMS_PATH = '/work/brian.matilla/WOFS_2021/MRMS/RAD_AZS_MSH_AGG/'
    
    wofs_t = t + WOFS_OFFSET
    mrms_t = t + MRMS_OFFSET
    mrms_begin_t =mrms_t - 6 
    try:
        mrms_files = [glob(join(MRMS_PATH, date, init_time, f'wofs_RAD_{tt:02d}*'))[0] for tt in 
                  range(mrms_begin_t, mrms_t+1)]
        wofs_file = glob(join(WOFS_PATH, date, init_time, f'wofs_ENSEMBLETRACKS_{wofs_t:02d}*'))[0]
        return mrms_files, wofs_file
    
    except IndexError:
        return None, None

## Identify tracks in the 30-min time-max MRMS Comp. Refl. 

In [4]:
def identify_mrms_tracks(mrms_files, min_thresh=44):
    """Identify the 30-min MRMS Compo. Refl. Tracks"""
    mrms_dbz = np.max([xr.load_dataset(f)['dz_cress'].values for f in mrms_files], axis=0)
    
    storm_labels, object_props = monte_python.label( input_data = mrms_dbz,
                                   method ='watershed', 
                                   return_object_properties=True, 
                                   params = {'min_thresh': min_thresh,
                                             'max_thresh': 75,
                                             'data_increment': 1,
                                              'area_threshold': 1500,
                                            'dist_btw_objects': 25 } )
    # Quality Control 
    qcer = monte_python.QualityControler()
    qc_params = [('min_area', 12), ('max_thresh', [45, 99])]
    qc_labels, qc_objects_props = qcer.quality_control(mrms_dbz, storm_labels, object_props, qc_params)
    
    return qc_labels, qc_objects_props

## Identify new ensemble storm tracks

In [5]:
def get_wofs_tracks(wofs_file):
    ds = xr.load_dataset(wofs_file)
    current_tracks = ds['w_up__ensemble_tracks'].values
    probs = ds['w_up__ensemble_probabilities'].values
    new_tracks, new_input_data, new_props = new_id(probs)
    current_props = regionprops(current_tracks)
    
    return current_tracks, new_tracks, current_props, new_props

## Match MRMS to the WoFS

In [6]:
def match(current_tracks, new_tracks, mrms_tracks):
    matcher = monte_python.ObjectMatcher(cent_dist_max = 20, 
                                     min_dist_max = 10,
                                     time_max=0, score_thresh=0.2, 
                                     one_to_one = False)

    matched_fcst, matched_obs, _ = matcher.match(ensemble_tracks, qc_labels)
    matched_new_fcst, matched_new_obs, _ = matcher.match(new_labels, qc_labels)

## Contingency Statistics 

In [7]:
dates = os.listdir(MRMS_PATH)[::2]
init_time = ['0000']
time_rng = range(0, 30, 2)

verifier_current = WoFSVerifier()
verifier_new = WoFSVerifier()

i = 0
for date in dates:
    if date[4:6] == '05':
        try:
            os.listdir(os.path.join(WOFS_PATH, date))
        except FileNotFoundError:
            continue
        for init_time in os.listdir(os.path.join(WOFS_PATH, date))[::3]:
            for t in time_rng:
                # Get MRMS and Ensemble storm tracks
                mrms_files, wofs_file = get_files(date, init_time, t)

   
                if wofs_file is None:
                    continue

                i+=1
                # Load WoFS Data and ID new tracks
                current_tracks, new_tracks, current_props, new_props = get_wofs_tracks(wofs_file)
            
                # Identify MRMS tracks 
                mrms_tracks, mrms_props = identify_mrms_tracks(mrms_files, min_thresh=44)
                
                # Match WOFS to MRMS
                matcher = monte_python.ObjectMatcher(cent_dist_max = 20, 
                                     min_dist_max = 10,
                                     time_max=0, score_thresh=0.2, 
                                     one_to_one = False)

                matched_fcst_current, matched_obs_current, _ = matcher.match(current_tracks, mrms_tracks)
                matched_fcst_new, matched_obs_new, _ = matcher.match(new_tracks, mrms_tracks)
            
     
                # Add hits, false alarms, and misses
                verifier_current.add_contingency_table_compos(current_tracks, 
                                                          mrms_tracks, matched_fcst_current, matched_obs_current)
            
                verifier_new.add_contingency_table_compos(new_tracks, 
                                                          mrms_tracks, matched_fcst_new, matched_obs_new)

In [8]:
pod, far, csi = verifier_current.get_scores()
pod_new, far_new, csi_new = verifier_new.get_scores()

N_old_storms = verifier_current.hits + verifier_current.false_alarms
N_new_storms = verifier_new.hits + verifier_new.false_alarms
N_mrms_storms = verifier_current.hits + verifier_current.misses

print(f'Number of Cases : {i}')
print('Scores for the existing tracks:\n')
print(f"""POD : {pod:.3f} | FAR : {far:.3f} | CSI : {csi:3f} | N : {N_old_storms}\n""")
print('Scores for the new tracks:\n')
print(f"""POD : {pod_new:.3f} | FAR : {far_new:.3f} | CSI : {csi_new:.3f} | N : {N_new_storms}\n""")
print(f"Num of MRMS Storms : {N_mrms_storms}")

Number of Cases : 1285
Scores for the existing tracks:

POD : 0.822 | FAR : 0.786 | CSI : 0.204166 | N : 34368

Scores for the new tracks:

POD : 0.688 | FAR : 0.520 | CSI : 0.394 | N : 12792

Num of MRMS Storms : 8935
